<a href="https://colab.research.google.com/github/OmdenaAI/ladakh-india-glacier-mapping/blob/main/src/visualizations/image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install folium
#pip install earthengine-api #if running on local conda environment
# initiate earthengine authenticate to start the process in the local conda environment
###only needed the first time your run
#ee.Authenticate() -----IMPORTANT

In [2]:
# Import necessary libraries
import ee
import folium




In [3]:
# Initialize the Earth Engine API




ee.Initialize()

# Define a method to display Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer



In [6]:
# Set the area of interest (AOI). For our example, it will be a rectangle around Ladakh
aoi = ee.Geometry.Rectangle([75.0, 36, 82, 32])

# Load a landsat image. Filter by date and AOI and cloud cover.
landsat = (ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
           .filterDate(ee.Date('2020-07-01'), ee.Date('2020-10-31'))
           .filterBounds(aoi)
            .filterMetadata('CLOUD_COVER', 'less_than', 10)
           .median())  # Take the median to remove clouds


# this was the process that was mentioned in one of the tutorials
# Use normalized difference to emphasize water bodies which can be indicative of glacial melt.
ndwi = landsat.normalizedDifference(['B3', 'B6'])

# Threshold the NDWI to create a binary mask of potential glacier locations.
glacier_mask = ndwi.gt(0.3)



In [7]:
# Visualize
location = [34.0, 77.5]  # For ladakh regions just a point somehwere in the rectabngle
m = folium.Map(location=location, zoom_start=7)

# Define the bounds of the rectangle. Format: [[lat_min, lon_min], [lat_max, lon_max]]
bounds = [[75, 36], [82, 32]]

# Add the rectangle to the map
folium.Rectangle(
    bounds=bounds,
    color='#ff7800',
    fill=True,
    fill_color='#ffff00',
    fill_opacity=0.2
).add_to(m)



# Add the raw image to the map
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}
m.add_ee_layer(landsat, vis_params, 'Landsat Image')

# Add the glacier mask to the map
m.add_ee_layer(glacier_mask.updateMask(glacier_mask), {'palette': 'blue'}, 'Glacier Mask')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)